# Exercise 2

Work on this before the next lecture on 24 April. We will talk about questions, comments, and solutions during the exercise after the third lecture.

Please do form study groups! When you do, make sure you can explain everything in your own words, do not simply copy&paste from others.

The solutions to a lot of these problems can probably be found with Google. Please don't. You will not learn a lot by copy&pasting from the internet.

If you want to get credit/examination on this course please upload your work to **your GitHub repository** for this course **before** the next lecture starts and post a link to your repository [in this thread](https://github.com/wildtreetech/advanced-comp-2017/issues/3). If you worked on things together with others please add their names to the notebook so we can see who formed groups.

---

These are some useful default imports for plotting and [`numpy`](http://www.numpy.org/)

In [1]:
%config InlineBackend.figure_format='retina'
%matplotlib inline

import numpy as np
np.random.seed(123)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams["font.size"] = 14
from sklearn.utils import check_random_state

## Question 1

Correlation between trees. This question is about investigating the correlation between decision trees and how this effects an ensemble constructed from them. There are three methods
for adding randomisation to the tree growing process:

1. grow each tree on a bootstrap sample
2. for each tree select a subset of features at random
3. pick the best random split point

You can use `RandomForestClassifier`, `BaggingClassifier`, and `ExtraTreesClassifier` to achieve various different sets of the above three strategies.

Show how the average amount of correlation between the trees in the ensemble varies as a function of bootstrap yes/no, number of `max_features`, and picking the best split point at random or not.

Pick one of the classification datasets from http://scikit-learn.org/stable/modules/classes.html#module-sklearn.datasets.

In [18]:
# your solution
from sklearn.datasets import load_digits
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from functools import reduce
def get_correlation(clf,X_test):
    avg_list = []
    for (ind,tree1) in enumerate(clf.estimators_):    
        [avg_list.append(np.corrcoef(tree1.predict(X_test),tree2.predict(X_test))[0][1]) for tree2 in clf.estimators_[ind + 1:]]
#    sum(np.corrcoef(tree1.predict(X_test),tree2.predict(X_test))[0][1] for tree2 in clf.estimators_[ind + 1:] for (ind,tree1) in enumerate(clf.estimators_))
    avg = reduce(lambda x,y: x+y, avg_list)/len(avg_list)
    return avg

X,y = load_digits(return_X_y=True)
#X,y = load_iris(return_X_y=True)
print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
clf = RandomForestClassifier(bootstrap = True, n_estimators=20).fit(X_train,y_train)
clf2 = RandomForestClassifier(bootstrap = False, n_estimators=20).fit(X_train,y_train)
#print(clf.estimators_[0])
clf3 = BaggingClassifier(bootstrap=True, max_features=20).fit(X_train,y_train)
clf4 = BaggingClassifier(bootstrap=False).fit(X_train,y_train)
clf5 = ExtraTreesClassifier(bootstrap=True).fit(X_train,y_train)
clf6 = ExtraTreesClassifier(bootstrap=False).fit(X_train,y_train)
random_forest_list = [RandomForestClassifier(bootstrap=True, n_estimators=20,max_features=x).fit(X_train,y_train) for x in range(5,65)]
bag_forest_list = [BaggingClassifier(bootstrap=True,n_estimators=20,max_features=x).fit(X_train,y_train) for x in range(5,65)]
extra_forest_list = [ExtraTreesClassifier(bootstrap=True, n_estimators=20,max_features=x).fit(X_train,y_train) for x in range(5,65)]
random_avg_corr_list_max_features = [get_correlation(x,X_test) for x in random_forest_list]
bag_avg_corr_list_max_features = [get_correlation(x,X_test) for x in bag_forest_list]
extra_avg_corr_list_max_features = [get_correlation(x,X_test) for x in extra_forest_list]
print('For Random Forests, Average Correlation with bootstrap is %f' % get_correlation(clf,X_test))
print('For Random Forests, Average Correlation without bootstrap is %f' % get_correlation(clf2,X_test))
print('For Bagging Forests, Average Correlation with bootstrap is %f' % get_correlation(clf3,X_test))
print('For Bagging Forests, Average Correlation without bootstrap is %f' % get_correlation(clf4,X_test))
print('For Extra Forests, Average Correlation with bootstrap is %f' % get_correlation(clf5,X_test))
print('For Extra Forests, Average Correlation without bootstrap is %f' % get_correlation(clf6,X_test))
plt.plot(range(5,65), random_avg_corr_list_max_features)
plt.plot(range(5,65), bag_avg_corr_list_max_features)
plt.plot(range(5,65), extra_avg_corr_list_max_features)
plt.xlabel('max_features')
plt.ylabel('Average correlation')
plt.legend(['Random','Bag','Extra'])

(1797, 64)


ValueError: Number of features of the model must match the input. Model n_features is 5 and input n_features is 64 

---

## Question 2

Compare the feature importances calculated by a `RandomForestClassifier`, `ExtraTreesClassifier` and `GradientBoostedTreesClassifier` on the digits dataset. You might have to tune `n_estimators` to get good performance. Which parts of the images is the most important and do you agree with the interpretation of the classifiers? (Bonus) Do the importances change if you change to problem to be a classification problem of odd vs even digit?

You can load the data set with: http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html#sklearn.datasets.load_digits

In [25]:
# your answer
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier,GradientBoostingClassifier

---

## Question 3

This is a regression problem. Use a gradient boosted tree regressor (tune the `max_depth`, `learning_rate` and `n_estimators` parameters) to study the importance of the different features as well as the partial dependence of the output on individual features as well as pairs of features.

* can you identify uninformative features?
* how do the interactions between the features show up in the partial dependence plots?

(Help: `rgr = GradientBoostingRegressor(n_estimators=200, max_depth=2, learning_rate=0.1)
` seems to work quite well)
(Help: to produce 1D and 2D partial dependence plots pass `[0,1, (0,1)]` as the `features` argument of `plot_partial_dependence`. More details in the function's documentation.)


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor


def make_data(n_samples=800, n_features=8, noise=0.2, random_state=2):
    generator = check_random_state(random_state)

    X = generator.rand(n_samples, n_features)
    y = 10 * (X[:, 0] * X[:, 1]) + 20 * (X[:, 2] - 0.5) ** 2 \
        + 10 * X[:, 3] + 10 * X[:, 4] + noise * generator.randn(n_samples)

    return X, y

X,y = make_data()

In [ ]:
# your solution

---

## (Bonus) Question 4

House prices in California. Use a gradient boosted regression tree model to build a model that can predict house prices in California (`GradientBoostingRegressor` is your friend).

Plot each of the features as a scatter plot with the target to learn about each variable. You can also make a plot of two features and use the target as colour.

Fit a model and tune the model complexity using a training and test data set.

Explore the feature importances and partial dependences that are important to the house price.

In [ ]:
from sklearn.datasets.california_housing import fetch_california_housing

cal_housing = fetch_california_housing()

# if the above doesn't work, download `cal_housing_py3.pkl` from the GitHub repository
# and adjust the path to the downloaded file which is passed to `load()`
# uncomment the following lines
#from sklearn.externals.joblib import load
#d = load('/home/username/Downloads/cal_housing_py3.pkz')
#X, y = d[:,1:], d[:,0]/100000
#X[:, 2] /= X[:, 5]
#X[:, 3] /= X[:, 5]
#X[:, 5] = X[:, 4] / X[:, 5]

In [ ]:
# your solution